# Importing Libraries and Preparing Dataset

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image  # Import PIL for image conversion

# Custom Dataset Class
class LOLVariationsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert NumPy array to PIL Image

        if self.transform:
            image = self.transform(image)

        return image

# Data Augmentation for Training Data with Resizing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 for reduced memory usage
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])

# Load Dataset and Split into Train and Validation Sets (80/20 split)
dataset_path = "/kaggle/input/lol-variations-like-sice"
dataset = LOLVariationsDataset(dataset_path, transform=transform)

train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)  # Reduced batch size and added num_workers
val_loader = DataLoader(val_data, batch_size=4, shuffle=False, num_workers=2)

# Zero-DIDCE Model

In [ ]:
import torch.nn as nn

# DiDCE-Net Model Definition
class DiDCENet(nn.Module):
    def __init__(self):
        super(DiDCENet, self).__init__()

        self.relu = nn.ReLU(inplace=True)

        # Convolutional layers with dropout for regularization
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(0.5)  # Dropout layer for regularization
        self.skip_conv = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))

        inverted_x = 1 - x
        x11 = self.relu(self.conv1(inverted_x))
        x21 = self.relu(self.conv2(x11))
        x31 = self.relu(self.conv3(x21))

        combined_features = torch.cat([x3, x31], dim=1)  # Concatenate features along the channel dimension
        combined_features = self.dropout(combined_features)  # Apply dropout
        curve_map = torch.tanh(self.skip_conv(combined_features))

        return curve_map

# Adaptive Light Enhancement (ALE) Curve

In [ ]:
def adaptive_light_enhancement(x_input, curve_map):
    alpha = 0.63  # Target light level
    mean_light_level = torch.mean(x_input).item()
    beta = -0.79 * mean_light_level**2 + 0.81 * mean_light_level + 1.41

    iterations_formula = (
        -25 * mean_light_level + 10 if mean_light_level < 0.1 else
        (17.14 * mean_light_level**2 - 15.14 * mean_light_level + 10 if mean_light_level < 0.45 else
         5.66 * mean_light_level**2 - 2.93 * mean_light_level + 7.2)
    )
    iterations = int(iterations_formula)

    enhanced_image = x_input.clone()  # Clone input to avoid modifying it directly
    for _ in range(iterations):
        enhanced_image = enhanced_image + curve_map * (enhanced_image**2 - enhanced_image) * \
                         ((alpha - torch.mean(enhanced_image).item()) / (beta - torch.mean(enhanced_image).item()))

    return enhanced_image

# Piecewise Non Reference Loss Function

In [ ]:
def piecewise_non_reference_loss(input_image, enhanced_image):
    Q1, Q2 = 0.2, 0.8
    E = 0.6

    mean_light_levels = torch.mean(enhanced_image.view(enhanced_image.size(0), -1), dim=1)

    L1_mask = (mean_light_levels <= Q1) | (mean_light_levels >= Q2)
    L2_mask = ~L1_mask

    L1_loss = ((mean_light_levels[L1_mask] - E)**2).mean() if L1_mask.any() else torch.tensor(0., device=enhanced_image.device)
    L2_loss = (((mean_light_levels[L2_mask] - E)**2) / (1 + mean_light_levels[L2_mask])).mean() if L2_mask.any() else torch.tensor(0., device=enhanced_image.device)

    W1, W2 = 0.5, 0.5
    loss_value = W1 * L1_loss + W2 * L2_loss

    return loss_value

# Evaluation Metrics

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

def evaluate_metrics(model, data_loader, device):
    model.eval()
    psnr_scores, ssim_scores = [], []

    with torch.no_grad():
        for images in data_loader:
            images = images.to(device)

            curve_map = model(images)
            enhanced_images = adaptive_light_enhancement(images.clone(), curve_map).cpu().numpy()

            for i in range(images.size(0)):
                original_img_np = images[i].cpu().numpy().transpose(1, 2, 0)
                enhanced_img_np = enhanced_images[i].transpose(1, 2, 0)

                psnr_scores.append(psnr(original_img_np, enhanced_img_np))
                ssim_scores.append(ssim(original_img_np,
                                        enhanced_img_np,
                                        win_size=3,
                                        data_range=1.0))  # Set data_range explicitly

    print(f"PSNR: {np.mean(psnr_scores)}, SSIM: {np.mean(ssim_scores)}")

# Training

In [ ]:
import torch.optim as optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model = DiDCENet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Learning rate scheduler for dynamic adjustment based on validation loss
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

for epoch in range(100):
    model.train()
    epoch_loss_train = 0

    # Training phase with tqdm progress bar
    with tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", unit="batch") as tepoch:
        for images in tepoch:
            images = images.to(device)

            # Forward pass through DiDCE-Net and ALE-Curve enhancement
            curve_map = model(images)
            enhanced_images = adaptive_light_enhancement(images.clone(), curve_map)

            # Compute loss and backpropagation
            loss_value = piecewise_non_reference_loss(images.clone(), enhanced_images.clone())
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()

            # Update epoch loss and progress bar
            epoch_loss_train += loss_value.item()
            tepoch.set_postfix(loss_train=loss_value.item())

    avg_train_loss = epoch_loss_train / len(train_loader)
    print(f"Epoch {epoch+1}, Average Train Loss: {avg_train_loss}")

    # Validation phase with tqdm progress bar
    model.eval()
    epoch_loss_val = 0

    with torch.no_grad():
        with tqdm(val_loader, desc=f"Epoch {epoch+1} [Validation]", unit="batch") as vepoch:
            for images in vepoch:
                images = images.to(device)

                curve_map_val = model(images)
                enhanced_images_val = adaptive_light_enhancement(images.clone(), curve_map_val)

                loss_value_val = piecewise_non_reference_loss(images.clone(), enhanced_images_val.clone())
                epoch_loss_val += loss_value_val.item()

    avg_val_loss = epoch_loss_val / len(val_loader)
    print(f"Epoch {epoch+1}, Average Validation Loss: {avg_val_loss}")

    # Step the scheduler based on validation loss to adjust learning rate dynamically.
    scheduler.step(avg_val_loss)

    # Evaluate metrics every five epochs.
    if (epoch + 1) % 5 == 0:
        print(f"--- Evaluation at Epoch {epoch+1} ---")
        evaluate_metrics(model, val_loader, device)

Epoch 1 [Train]: 100%|██████████| 582/582 [00:12<00:00, 46.00batch/s, loss_train=0.00422] 


Epoch 1, Average Train Loss: 0.140533632018841


Epoch 1 [Validation]: 100%|██████████| 146/146 [00:01<00:00, 109.16batch/s]


Epoch 1, Average Validation Loss: 0.13957707024153884


Epoch 2 [Train]: 100%|██████████| 582/582 [00:12<00:00, 47.14batch/s, loss_train=0.131]  


...

Epoch 100 [Train]: 100%|██████████| 582/582 [00:12<00:00, 46.45batch/s, loss_train=0.0118] 


Epoch 100, Average Train Loss: 0.1429678655701171


Epoch 100 [Validation]: 100%|██████████| 146/146 [00:01<00:00, 107.36batch/s]

Epoch 100, Average Validation Loss: 0.14168404617461644
--- Evaluation at Epoch 100 ---


PSNR: 22.5141660160638, SSIM: 0.5538395026455541


# Saving the model

In [ ]:
# Assuming `model` is your trained model
model_save_path = "/kaggle/working/zero_didce_model.pth"
torch.save(model.state_dict(), model_save_path)

In [ ]:
# Save the entire model
torch.save(model, "/kaggle/working/zero_didce_entire_model.pth")